In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense,Dropout,Activation,Input
from keras.models import Sequential,Model
from keras import metrics
import matplotlib.pyplot as plt
import os

Using TensorFlow backend.


In [2]:
#from grf file get data
def get_data(filename):
    df = pd.read_table(filename)
# def get_data():
#     df = pd.read_table('data/02_052A.grf')
    t = df.loc[df['DPLOT v1.6'].str.contains(r'\s+[0]$')].index.tolist()
    arr = df.values
    data_class = int(df.iat[1, 0])
    data_name = []
    normalised_x = []
    normalised_y = []
    for i in range(data_class):
        data_x = []
        data_y = []
        data_name.append(df.iat[int(t[i])+1,0])
        if i == 0:
            x1 = df[3:int(t[i])].values
            for j in range(x1.shape[0]):
                data_x.append(''.join(x1[j]).split(',')[0])
                data_y.append(''.join(x1[j]).split(',')[1])
        else:
            x2 = df[int(t[i-1])+3:int(t[i])].values
            for k in range(x2.shape[0]):
                data_x.append(''.join(x2[k]).split(',')[0])
                data_y.append(''.join(x2[k]).split(',')[1])
        normalised_x.append(np.array(data_x))
        normalised_y.append(np.array(data_y))
#     print('2222',normalised_x,"333",normalised_y)
    data_class = str(df.iat[t[-1]+5, 0])
    if "Pressure" in data_class:
        data_class = 'Pressure'
    else:
        data_class = 'Impulses'
    title = df.iat[t[-1]+3,0].replace("(","").replace(")","")
    N = title.split(', ')[0].split(' = ')[1]
    lL = title.split(', ')[1].split(' = ')[1]
    hH = title.split(', ')[2].split(' = ')[1]
    LH = title.split(', ')[3].split(' = ')[1]
    return data_class, N, lL, hH, LH, np.array(data_name),np.array(normalised_x),np.array(normalised_y)

In [3]:
#model create
def make_model():
    model=Sequential()
    model.add(Dense(units=100,activation='relu',input_dim=2))
#     model.add(Dropout(0.05))
    model.add(Dense(units=80,activation='relu'))
    model.add(Dense(units=40,activation='relu'))
#     model.add(Dropout(0.05))
    model.add(Dense(units=1,activation=None))
    model.compile(loss='mse',optimizer='adam',metrics=[metrics.mae])
    print(model.summary())
    return model
model = make_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_2 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 41        
Total params: 11,661
Trainable params: 11,661
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
def processer_data(data_class, N, lL, hH, LH, data_name, normalised_x, normalised_y):
    if data_class == 'Pressure':
        for i in range(len(data_name)):
            trainp = []
            for j in range(len(normalised_x[i])):
                trainp.append([N, lL, hH, LH, data_name[i]])
            trainp = np.array(trainp).reshape(-1, 5)
            xp.append(trainp)
        xp = np.array(xp).reshape(-1,5)
        yp = np.array(normalised_y).reshape(-1,1)
#             print(np.shape(xp), xp,np.shape(yp),yp)
    else:
        for i in range(len(data_name)):
            traini = []
            for j in range(len(normalised_x[i])):
                traini.append([N, lL, hH, LH, data_name[i]])
            traini = np.array(traini).reshape(-1, 5)
            xi.append(traini)
        xi = np.array(xi).reshape(-1,5)
        yi = np.array(normalised_y).reshape(-1,1)
#         print(np.shape(x), x,np.shape(y),y)
    pressure_x.append(xp)
    pressure_y.append(yp)
    impulses_x.append(xi)
    impulses_y.append(yi)

In [ ]:
#Traverse all grf files in folder
#Process data to train data
dataDir = './data'
pressure_x = []
pressure_y = []
impulses_x = []
impulses_y = []
for dirName, subdirList, fileList in os.walk(dataDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        xi = []
        yi = []
        xp = []
        yp = []
        filename = str(dirName + '/' + fname)
        data_class, N, lL, hH, LH, data_name, normalised_x, normalised_y = get_data(filename)
        if data_class == 'Pressure':
            for i in range(len(data_name)):
                trainp = []
                for j in range(len(normalised_x[i])):
                    trainp.append([N, lL, hH, LH, data_name[i]])
                trainp = np.array(trainp).reshape(-1, 5)
                xp.append(trainp)
            xp = np.array(xp).reshape(-1,5)
            yp = np.array(normalised_y).reshape(-1,1)
#             print(np.shape(xp), xp,np.shape(yp),yp)
        else:
            for i in range(len(data_name)):
                traini = []
                for j in range(len(normalised_x[i])):
                    traini.append([N, lL, hH, LH, data_name[i]])
                traini = np.array(traini).reshape(-1, 5)
                xi.append(traini)
            xi = np.array(xi).reshape(-1,5)
            yi = np.array(normalised_y).reshape(-1,1)
    #         print(np.shape(x), x,np.shape(y),y)
        pressure_x.append(xp)
        pressure_y.append(yp)
        impulses_x.append(xi)
        impulses_y.append(yi)
#         print(np.shape(pressure_x), pressure_x,np.shape(pressure_y),pressure_y)
    pressure_x_train = np.array(pressure_x).reshape(-1,5)
    pressure_y_train = np.array(pressure_y).reshape(-1,1)
    impulses_x_train = np.array(impulses_x).reshape(-1,5)
    impulses_y_train = np.array(impulses_y).reshape(-1,1)
    print(np.shape(pressure_x_train),pressure_x_train,np.shape(pressure_y_train),pressure_y_train)

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)
model.save_weights('modules/module.h5')

In [226]:
model.load_weights('modules/module.h5',by_name=False)
x_test = np.array([[1, 1]])
pred=model.predict(x_test)
print(pred)

[[ 4417.6875]]
